In [65]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving insta_comments.csv to insta_comments (1).csv
Saving lung_cancer_examples.csv to lung_cancer_examples.csv
User uploaded file "insta_comments.csv" with length 445205 bytes
User uploaded file "lung_cancer_examples.csv" with length 1704 bytes


In [3]:
import random
import re
import io

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

from tensorflow import keras

In [75]:
names = ["Age", "Smokes", "AreaQ", "Alkhol", "Result"]
df = pd.read_csv(io.StringIO(uploaded['lung_cancer_examples.csv'].decode('utf-8')))
df = df.drop(df.columns[[0, 1]], axis=1)
df = df.fillna(df.mean())
df.head()

,Age,Smokes,AreaQ,Alkhol,Result
0,35,3,5,4,1
1,27,20,2,5,1
2,30,0,5,2,0
3,28,0,8,1,0
4,68,4,5,6,1


In [76]:
# 1
for column in names[1:]:
    df[column] = (df[column] - df[column].mean())/df[column].std()
Y = df.iloc[:, 0].values.reshape(-1, 1)
Y = OneHotEncoder().fit_transform(Y).toarray()
X = df.iloc[:, 1:].values
num_samples = X.shape[0]
train_samples = int(0.75*num_samples)
indexes = np.arange(num_samples)
random.shuffle(indexes)
X, Y = X[indexes], Y[indexes]
X_train, Y_train = X[:train_samples, :], Y[:train_samples]
X_test, Y_test = X[train_samples:, :], Y[train_samples:]
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(44, 4) (44, 37)
(15, 4) (15, 37)


In [82]:
inputs = keras.Input(shape=(4,))
x = keras.layers.Dense(64, activation="relu")(inputs)
outputs = keras.layers.Dense(37, activation="softmax")(x)
fc_model = keras.Model(inputs=inputs, outputs=outputs, name="fc_model")
fc_model.summary()

Model: "fc_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_32 (Dense)             (None, 64)                320       
_________________________________________________________________
dense_33 (Dense)             (None, 37)                2405      
Total params: 2,725
Trainable params: 2,725
Non-trainable params: 0
_________________________________________________________________


In [83]:
fc_model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)
fc_model.fit(X_train, Y_train, batch_size=12, epochs=10, validation_split=0.2)

Epoch 1/10
3/3 [==============================] - 1s 181ms/step - loss: 3.5848 - accuracy: 0.0911 - val_loss: 3.7071 - val_accuracy: 0.0000e+00
Epoch 2/10
3/3 [==============================] - 0s 20ms/step - loss: 3.5013 - accuracy: 0.0741 - val_loss: 3.7242 - val_accuracy: 0.0000e+00
Epoch 3/10
3/3 [==============================] - 0s 21ms/step - loss: 3.4645 - accuracy: 0.0949 - val_loss: 3.7392 - val_accuracy: 0.0000e+00
Epoch 4/10
3/3 [==============================] - 0s 21ms/step - loss: 3.3810 - accuracy: 0.1158 - val_loss: 3.7536 - val_accuracy: 0.0000e+00
Epoch 5/10
3/3 [==============================] - 0s 20ms/step - loss: 3.3644 - accuracy: 0.1301 - val_loss: 3.7681 - val_accuracy: 0.0000e+00
Epoch 6/10
3/3 [==============================] - 0s 21ms/step - loss: 3.3517 - accuracy: 0.0988 - val_loss: 3.7829 - val_accuracy: 0.0000e+00
Epoch 7/10
3/3 [==============================] - 0s 24ms/step - loss: 3.2950 - accuracy: 0.1613 - val_loss: 3.7977 - val_accuracy: 0.0000e+0

In [85]:
test = fc_model.evaluate(X_test, Y_test, verbose=2)
print("Loss test:", test[0])
print("Accuracy test:", test[1])

1/1 - 0s - loss: 3.8944 - accuracy: 0.0667
Loss test: 3.894390344619751
Accuracy test: 0.06666667014360428


In [87]:
# 2
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
print(x_train.shape)
inputs = keras.Input(shape=(28, 28, 1))
x = keras.layers.Conv1D(64, 3, activation="softmax")(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Flatten()(x)
outputs = keras.layers.Dense(10)(x)
cnn_model = keras.Model(inputs=inputs, outputs=outputs, name="cnn_model")
cnn_model.summary()

(60000, 28, 28)
Model: "cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_65 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 28, 26, 64)        256       
_________________________________________________________________
batch_normalization_25 (Batc (None, 28, 26, 64)        256       
_________________________________________________________________
flatten_25 (Flatten)         (None, 46592)             0         
_________________________________________________________________
dense_25 (Dense)             (None, 10)                465930    
Total params: 466,442
Trainable params: 466,314
Non-trainable params: 128
_________________________________________________________________


In [88]:
cnn_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)
cnn_model.fit(x_train, y_train, batch_size=256, epochs=3, validation_split=0.2)

Epoch 1/3
188/188 [==============================] - 89s 471ms/step - loss: 2.1149 - accuracy: 0.7166 - val_loss: 3.4916 - val_accuracy: 0.1643
Epoch 2/3
188/188 [==============================] - 83s 443ms/step - loss: 0.7913 - accuracy: 0.8451 - val_loss: 0.9934 - val_accuracy: 0.6780
Epoch 3/3
188/188 [==============================] - 84s 447ms/step - loss: 0.4949 - accuracy: 0.8828 - val_loss: 0.6735 - val_accuracy: 0.8191


In [89]:
test = cnn_model.evaluate(x_test, y_test, verbose=2)
print("Loss test:", test[0])
print("Accuracy test:", test[1])

313/313 - 4s - loss: 0.7149 - accuracy: 0.8084
Loss test: 0.7148505449295044
Accuracy test: 0.8083999752998352


In [91]:
# 3
df = pd.read_csv(io.StringIO(uploaded['insta_comments.csv'].decode('utf-8')))
df = df.fillna(df.mean())
df.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [92]:
import nltk
nltk.download('stopwords')

def remove_stopwords(str_x):
    words = str_x.split(' ')
    neutral_words = ['edit', 'got', 'thing', 'want', 'make', 'is', 'do']
    new_words = list()
    for word in words:
        if word not in stopwords.words('english') or word not in neutral_words:
            new_words.append(word)
    return ' '.join(new_words)

df["comment_text"] = df["comment_text"].map(lambda x: re.sub(r'[^\w]', ' ', x))
df["comment_text"] = df["comment_text"].map(lambda x: x.lower())
df["comment_text"] = df["comment_text"].map(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [93]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["comment_text"].values).toarray()
Y = df.iloc[:, 3:].values
num_samples = X.shape[0]
train_samples = int(0.75*num_samples)
indexes = np.arange(num_samples)
random.shuffle(indexes)
X, Y = X[indexes], Y[indexes]
x_train, y_train = X[:train_samples, :], Y[:train_samples]
x_test, y_test = X[train_samples:, :], Y[train_samples:]
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(750, 9372) (750, 6)
(250, 9372) (250, 6)


In [94]:
inputs = keras.Input(shape=(9372, 1))
x = keras.layers.LSTM(128)(inputs)
outputs = keras.layers.Dense(6, activation="softmax")(x)
rnn_model = keras.Model(inputs=inputs, outputs=outputs, name="rnn_model")
rnn_model.summary()

Model: "rnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, 9372, 1)]         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_35 (Dense)             (None, 6)                 774       
Total params: 67,334
Trainable params: 67,334
Non-trainable params: 0
_________________________________________________________________


In [95]:
rnn_model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

rnn_model.fit(x_train, y_train, batch_size=4, epochs=1, validation_split=0.2)

150/150 [==============================] - 747s 5s/step - loss: 0.3803 - accuracy: 0.9372 - val_loss: 0.2816 - val_accuracy: 1.0000


In [96]:
test = rnn_model.evaluate(x_test, y_test, verbose=2)
print("Loss test:", test[0])
print("Accuracy test:", test[1])

8/8 - 16s - loss: 0.4472 - accuracy: 0.9960
Loss test: 0.4471825659275055
Accuracy test: 0.9959999918937683
